# ДЗ 8: Fine-tuning + LangChain Tools

**Трек C — Инструктивный помощник**

1. Fine-tuning на FineTome-100k (следование инструкциям)
2. LangChain Tools: text_formatter, template_generator, content_validator
3. Интеграция: fine-tuned модель + tools в агенте

## 0. Установка зависимостей

> ⚠️ В Colab после `pip install` может понадобиться **Runtime → Restart session**.

In [1]:
# Порядок установки: transformers + huggingface_hub должны быть совместимы (is_offline_mode)
# langchain-huggingface НЕ ставим — он требует hf_hub<1.0, а transformers 5.x нужен >=1.3
!pip install -q -U huggingface_hub>=1.3.0 transformers datasets peft accelerate bitsandbytes trl
!pip install -q langchain langchain-core langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


## 1. Fine-tuning (FineTome-100k + LoRA)

**Выбор модели:** `tiny` — быстрый показ (~1 ч, публичная), `mistral` — качественнее (~5 ч).

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# tiny = ~1 ч (Colab T4, публичная), mistral = ~5 ч, качественнее
MODEL_MODE = "tiny"  # "tiny" | "mistral"

MODELS = {
    "tiny": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",   # 1.1B, публичная, быстрый
    "mistral": "mistralai/Mistral-7B-v0.1",
}
model_id = MODELS[MODEL_MODE]

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

print(f"Модель: {model_id}")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)
model = prepare_model_for_kbit_training(model)

Модель: TinyLlama/TinyLlama-1.1B-Chat-v1.0


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079


In [4]:
# Подготовка FineTome-100k → instruction format
from datasets import load_dataset

ds = load_dataset("mlabonne/FineTome-100k", split="train")
# Размер зависит от модели: smol — 800 (~1 ч), mistral — 1500 (~5 ч)
N_SAMPLES = 800 if MODEL_MODE == "tiny" else 1500
ds = ds.select(range(N_SAMPLES))

def format_conversation(sample):
    conv = sample["conversations"]
    if len(conv) < 2:
        return None
    inst = next((m["value"] for m in conv if m.get("from") == "human"), "")
    resp = next((m["value"] for m in conv if m.get("from") == "gpt"), "")
    if not inst or not resp:
        return {"text": ""}
    text = f"""### Instruction:
{inst}

### Response:
{resp}"""
    return {"text": text}

ds = ds.map(format_conversation, remove_columns=ds.column_names)
ds = ds.filter(lambda x: x["text"] and len(x["text"]) > 10)
print(ds.num_rows, "примеров")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Filter:   0%|          | 0/800 [00:00<?, ? examples/s]

800 примеров


In [5]:
from trl import SFTTrainer, SFTConfig

# T4: bf16 недоступен, fp16 даёт BFloat16/scaler конфликт → fp32 без AMP
try:
    cap = torch.cuda.get_device_capability() if torch.cuda.is_available() else (0, 0)
    use_bf16 = cap[0] >= 8
except Exception:
    use_bf16 = False
# На T4 fp16 даёт BFloat16/scaler конфликт → отключаем AMP
use_amp = use_bf16
print("Precision: fp32" if not use_amp else f"Precision: {'bf16' if use_bf16 else 'fp16'}")

trainer = SFTTrainer(
    model=model,
    train_dataset=ds,
    processing_class=tokenizer,
    args=SFTConfig(
        max_length=512,
        per_device_train_batch_size=2 if not use_amp else (8 if MODEL_MODE == "tiny" else 4),
        gradient_accumulation_steps=8 if not use_amp else 4,
        warmup_steps=15,
        max_steps=80 if MODEL_MODE == "tiny" else 120,
        learning_rate=2e-4,
        bf16=use_bf16 if use_amp else False,
        fp16=(not use_bf16 and use_amp),
        logging_steps=5,
        logging_first_step=True,
        logging_strategy="steps",
        report_to="none",
        output_dir="outputs_finetome",
        optim="paged_adamw_8bit",
        save_steps=40,
        save_total_limit=2,
        run_name="finetome_lora"
    ),
)

n_steps = 80 if MODEL_MODE == "tiny" else 120
eta = "~1 ч" if MODEL_MODE == "tiny" else "~4–5 ч"
print(f"Датасет: {len(ds)} примеров | Шагов: {n_steps} | {eta}")
print("Запуск обучения...")
result = trainer.train()
print(f"\nГотово. Loss: {result.training_loss:.4f} | Время: {result.metrics.get('train_runtime', 0):.0f} сек")
trainer.save_model("outputs_finetome/final")
tokenizer.save_pretrained("outputs_finetome/final")
print("Модель сохранена в outputs_finetome/final")

Precision: fp32


Adding EOS to train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2051 > 2048). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Датасет: 800 примеров | Шагов: 80 | ~1 ч
Запуск обучения...


Step,Training Loss
1,1.323386
5,1.254041
10,1.231141
15,1.160697
20,1.182596
25,1.141263
30,1.093851
35,1.058530
40,1.099480
45,1.098622



Готово. Loss: 1.1088 | Время: 642 сек
Модель сохранена в outputs_finetome/final


## 2. LangChain Tools

In [6]:
from langchain_core.tools import tool


@tool
def text_formatter(text: str, width: int = 80, indent: int = 2) -> str:
    """Форматирует текст: перенос по width, отступ indent. Вход: text (строка), width (макс. символов в строке), indent (отступ)."""
    lines = text.replace("\n", " ").split()
    result = []
    current = ""
    prefix = " " * indent
    for w in lines:
        if len(current) + len(w) + 1 <= width:
            current = f"{current} {w}".strip() if current else w
        else:
            if current:
                result.append(prefix + current)
            current = w
    if current:
        result.append(prefix + current)
    return "\n".join(result) if result else text


@tool
def template_generator(template_type: str, placeholders: str = "") -> str:
    """Генерирует шаблон по типу. template_type: 'email'|'json'|'markdown'|'prompt'. placeholders — список полей через запятую."""
    templates = {
        "email": "Subject: {subject}\n\nDear {name},\n\n{body}\n\nBest regards,\n{sender}",
        "json": "{\"key\": \"value\"}",
        "markdown": "# {title}\n\n## Введение\n{intro}\n\n## Основная часть\n{content}\n\n## Заключение\n{conclusion}",
        "prompt": "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    }
    return templates.get(template_type.lower(), templates["prompt"])


@tool
def content_validator(text: str, rules: str = "structure") -> str:
    """Проверяет текст по правилам. rules: 'structure' (есть заголовки/параграфы) | 'length' (не пустой) | 'format' (markdown/email). Возвращает JSON: valid (bool), issues (list)."""
    import json
    issues = []
    if rules == "length" or "length" in rules:
        if not text or not text.strip():
            issues.append("Текст пустой")
    if rules == "structure" or "structure" in rules:
        if "#" not in text and "\n\n" not in text:
            issues.append("Нет чёткой структуры (заголовки/параграфы)")
    if rules == "format" or "format" in rules:
        if "@" in text and "Subject:" not in text:
            issues.append("Похоже на email, но нет Subject")
    return json.dumps({"valid": len(issues) == 0, "issues": issues}, ensure_ascii=False)

In [7]:
tools = [text_formatter, template_generator, content_validator]
for t in tools:
    print(t.name, ":", t.description[:60] + "...")

text_formatter : Форматирует текст: перенос по width, отступ indent. Вход: te...
template_generator : Генерирует шаблон по типу. template_type: 'email'|'json'|'ma...
content_validator : Проверяет текст по правилам. rules: 'structure' (есть заголо...


## 3. Интеграция: fine-tuned модель + tools

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline
from peft import AutoPeftModelForCausalLM

# Загрузка fine-tuned адаптера (или базовой модели, если обучение ещё не запущено)
adapter_path = "outputs_finetome/final"
model_id = "mistralai/Mistral-7B-v0.1"

try:
    tokenizer = AutoTokenizer.from_pretrained(adapter_path)
    model = AutoPeftModelForCausalLM.from_pretrained(
        adapter_path,
        device_map="auto",
        torch_dtype=torch.float16
    )
except Exception:
    # Fallback: базовая модель (если адаптер не сохранён)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline=pipe)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Passing `generation_config` together with generation-related arguments=({'temperature', 'do_sample', 'pad_token_id', 'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
/tmp/ipython-input-3681899756.py:32: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [9]:
# Mistral не поддерживает tool calling нативно — используем прямые вызовы tools
# ReAct-агент доступен, но Mistral без tool-calling может давать нестабильный вывод.
# Используем прямые вызовы tools (см. ячейку ниже) как основной демо-сценарий.

In [10]:
# Демо: прямой вызов модели (без tools)
def trim_response(text):
    """Обрезаем, если модель начала новый блок Instruction."""
    for stop in ["\n\n### Instruction", "### Instruction", "\n### Response"]:
        if stop in text:
            text = text.split(stop)[0]
    return text.strip()

def generate(prompt_text, max_new_tokens=200):
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    raw = tokenizer.decode(out[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return trim_response(raw)

user_prompt = "Объясни, что такое рекурсия в программировании. Приведи короткий пример на Python."
full_prompt = f"""### Instruction:
{user_prompt}

### Response:
"""

print("Ответ модели:")
print(generate(full_prompt))

Ответ модели:
Reкурсия является процедурным выполнением функции, которая уже выполнена. В Python рекурсия используется для решения задач, где ответ может быть получен только после выполнения некоторого кода. В программировании рекурсия представляет собой процедуру, которая выполняется с помощью других процедур. В Python рекурсия используется для решения задач, которые возникают в процессе работы с данными.

В примере ниже представлено напиток на Python. В этом примере рекурсией является функция `square`, которая принимает число `num` и возвращает его в разнуздующем виде. Функция `square` рекурсивно выполняется до тех пор, пока число `num` будет больше или равно `2`. После этого приращение `num`


In [11]:
# Демо: агент с tools (если модель поддерживает ReAct)
# Для Mistral без tool calling может работать нестабильно — альтернатива: ручной вызов tools

# Прямой вызов tools:
sample = "Длинный текст который нужно отформатировать по ширине 40 символов с отступом 2 пробела для читаемости."
print("text_formatter:", text_formatter.invoke({"text": sample, "width": 40, "indent": 2}))
print()
print("template_generator:", template_generator.invoke({"template_type": "prompt"}))
print()
print("content_validator:", content_validator.invoke({"text": "# Заголовок\n\nПараграф.", "rules": "structure"}))

text_formatter:   Длинный текст который нужно
  отформатировать по ширине 40 символов с
  отступом 2 пробела для читаемости.

template_generator: ### Instruction:
{instruction}

### Input:
{input}

### Response:


content_validator: {"valid": true, "issues": []}


## 4. Опционально: системный промпт «когнитивный дизайнер»

Для объяснений в стиле когнитивного дизайнера — см. `prompt_cognitive_designer.md` или `../06_prompting_guide/promt.md`. Добавь его в `full_prompt` перед запросом пользователя.

In [12]:
# Пример с системным промптом-когнитивным дизайнером
# Инструкция — в формате, который модель знает: внутри Instruction
user_q = "Объясни, что такое замыкание в JavaScript."
instruction = f"Стиль: объясняй через аналогии из жизни, начни с «Представь...». Вопрос: {user_q}"
prompt_cognitive = f"""### Instruction:
{instruction}

### Response:
"""

print("С когнитивным дизайнером:")
print(generate(prompt_cognitive, max_new_tokens=250))

С когнитивным дизайнером:
Разработка программного обеспечения на языке JavaScript и решение проблем, связанных с ней, заключаются в том, чтобы определить, как работают эти программы и как их интерпретируют. Определение термина "замыкание" в JavaScript обязательно смотреть в сторону и почитать.

Предположим, что нам нужно создать функцию, которая возвращает значение 25. Это можно сделать следующим образом:

```javascript
function getNumber() {
  return 25;
}
```

Этот код создаёт функцию, которая возвращает значение 25. Функция "getNumber" состоит из двух частей. Первая часть вызывает метод "return" (переменная), который возвращает значение 25. Во втором блоке кода вызываются инициализирующие операции, которые подсчитывают значение 25 и выводят его в кон
